In [10]:
import pygmtools as pygm
import numpy as np 
import networkx as nx 
import itertools


pygm.set_backend('numpy')
np.random.seed(0)


def to_matrix(D, ln):
    results = np.zeros((ln,ln))
    for i in D: 
        for j in i[1]:
            results[int(i[0]),int(j)] = i[1][j]
    return results 


def topk_from_dict(scores_dict,k ):
    score_tuples = sorted([ (i, scores_dict[i]) for i in scores_dict], key= lambda x : x[1], reverse=True)
    top_k = [i[0] for i in score_tuples[:k]]
    return top_k


def find_landmarks(G,k):
    centrals = nx.betweenness_centrality(G)
    return topk_from_dict(centrals, k)

def generate_scores(G_anon, G_pub, lands_anon, lands_pub):
    n1 = len(G_anon)
    n2 = len(G_pub)
    D_anon = nx.all_pairs_shortest_path_length(G_anon)
    D_pub = nx.all_pairs_shortest_path_length(G_pub)
    D_anon = to_matrix(D_anon, n1)
    D_pub = to_matrix(D_pub, n2)
    weights = np.zeros((n1, n2))
    for i in range(n1):
        if(i in lands_anon): 
            continue
        for j in range(n2):
            if(j in lands_pub): 
                continue
            score = 0
            if(len(lands_anon) != len(lands_pub)):
                print("hey")
            for k in range(len(lands_anon)):
                d1 = D_anon[i][int(lands_anon[k])]
                d2 = D_pub[j][int(lands_pub[k])]
                score += (d1 - d2)**2
            weights[i,j] = -np.sqrt(score) 
    for i in range(len(lands_anon)):
        weights[lands_anon[i], :] = -99999
        weights[:, lands_pub[i]] = -99999
        weights[lands_anon[i], lands_pub[i]] = 100 
    return weights

def find_matching(weights, lands_anon, lands_pub):
    matching = pygm.hungarian(weights)
    score = 0 
    for edge in matching:
        a = int(edge[0])
        b = int(edge[1])
        score += weights[a,b]
    return matching, score 



def distance_vector_method(G_anon, G_pub):
    k = 40
    k_iters = 1


    best_score = np.Inf 
    best_matching = None 
    best_perm = None 
    lands_anon = [i for i in range(40)]
    lands_pub = [i for i in range(40)]
    #lands_anon = find_landmarks(G_anon,k)
    #lands_pub = find_landmarks(G_pub,k)
    c = 0
    for perm in itertools.permutations([i for i in range(k)]):
        c += 1
        mapping_anon = [i for i in lands_anon] ## this is always unshuffled 
        mapping_pub = [lands_pub[i] for i in perm]
        W  = generate_scores(G_anon, G_pub, mapping_anon, mapping_pub)
        match, score = find_matching(W, mapping_anon, mapping_pub)
        if(score < best_score):
            best_score = score 
            best_matching = match
            best_perm = perm 
        if(c >= k_iters): break 
    matching = [(i, np.argmax(best_matching[i,:])) for i in range(len(best_matching))]
    return matching




In [ ]:
import defenses.rw as rw

In [3]:
G = nx.read_edgelist("data/email-Eu-core.txt", nodetype=int)

In [4]:
mapp = [i for i in range(40)]

In [5]:
G_rw = rw.Anon(G, [5,40])[0]


In [11]:
distance_vector_method(G, G_rw)

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10),
 (11, 11),
 (12, 12),
 (13, 13),
 (14, 14),
 (15, 15),
 (16, 16),
 (17, 17),
 (18, 18),
 (19, 19),
 (20, 20),
 (21, 21),
 (22, 22),
 (23, 23),
 (24, 24),
 (25, 25),
 (26, 26),
 (27, 27),
 (28, 28),
 (29, 29),
 (30, 30),
 (31, 31),
 (32, 32),
 (33, 33),
 (34, 34),
 (35, 35),
 (36, 36),
 (37, 37),
 (38, 38),
 (39, 39),
 (40, 40),
 (41, 486),
 (42, 932),
 (43, 894),
 (44, 161),
 (45, 738),
 (46, 300),
 (47, 437),
 (48, 908),
 (49, 719),
 (50, 224),
 (51, 114),
 (52, 591),
 (53, 320),
 (54, 131),
 (55, 57),
 (56, 803),
 (57, 59),
 (58, 56),
 (59, 484),
 (60, 344),
 (61, 564),
 (62, 107),
 (63, 405),
 (64, 86),
 (65, 103),
 (66, 397),
 (67, 551),
 (68, 513),
 (69, 371),
 (70, 368),
 (71, 952),
 (72, 934),
 (73, 97),
 (74, 74),
 (75, 76),
 (76, 980),
 (77, 156),
 (78, 918),
 (79, 292),
 (80, 303),
 (81, 174),
 (82, 533),
 (83, 420),
 (84, 116),
 (85, 537),
 (86, 84),
 (87, 147),
 (88, 235),
 